In [ ]:
import pandas as pd
from gensim.models.word2vec import Word2Vec
from random import shuffle
import copy

In [ ]:
train = pd.read_pickle('data/train_bo2.pkl')
test = pd.read_pickle('data/test_bo2.pkl')
data = pd.concat([train, test], sort=True)

In [ ]:
data.shape

In [ ]:
features = ["FLAG_OWN_CAR","FLAG_OWN_REALTY","NAME_TYPE_SUITE","NAME_INCOME_TYPE","NAME_EDUCATION_TYPE","NAME_FAMILY_STATUS","NAME_HOUSING_TYPE","OCCUPATION_TYPE","FLAG_DOCUMENT_2","FLAG_DOCUMENT_3","FLAG_DOCUMENT_4","FLAG_DOCUMENT_5","FLAG_DOCUMENT_6","FLAG_DOCUMENT_7","FLAG_DOCUMENT_8","FLAG_DOCUMENT_9","FLAG_DOCUMENT_10","FLAG_DOCUMENT_11","FLAG_DOCUMENT_12", "FLAG_DOCUMENT_13","FLAG_DOCUMENT_14","FLAG_DOCUMENT_15","FLAG_DOCUMENT_16","FLAG_DOCUMENT_17","FLAG_DOCUMENT_18", "FLAG_DOCUMENT_19","FLAG_DOCUMENT_20","FLAG_DOCUMENT_21", "CAT_MONTHS_BIRTH", 'CNT_CHILDREN', 'CODE_GENDER', 'CONTACT', 'EMERGENCYSTATE_MODE', 'FLAG_EMAIL', 'FLAG_OTHER', 'FONDKAPREMONT_MODE', 'HOUR_APPR_PROCESS_START', 'HOUSETYPE_MODE', 'LIVE_CITY_NOT_WORK_CITY', 'LIVE_REGION_NOT_WORK_REGION', 'NAME_CONTRACT_TYPE', 'ORGANIZATION_TYPE', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'REGION_RATING_CLIENT_W_CITY_times_livingsum_AVG', 'REGION_RATING_CLIENT_W_CITY_times_livingsum_MEDI', 'REGION_RATING_CLIENT_W_CITY_times_livingsum_MODE', 'REGION_RATING_CLIENT_times_livingsum_AVG', 'REGION_RATING_CLIENT_times_livingsum_MEDI', 'REGION_RATING_CLIENT_times_livingsum_MODE', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'WALLSMATERIAL_MODE', 'WEEKDAY_APPR_PROCESS_START', 'count_CODE_GENDER_NAME_EDUCATION_TYPE', 'count_CODE_GENDER_NAME_EDUCATION_TYPE_NAME_INCOME_TYPE', 'count_CODE_GENDER_NAME_INCOME_TYPE', 'count_NAME_EDUCATION_TYPE_NAME_INCOME_TYPE', 'unique_CODE_GENDER_NAME_EDUCATION_TYPE', 'unique_CODE_GENDER_NAME_INCOME_TYPE', 'unique_CODE_GENDER_x', 'unique_CODE_GENDER_y', 'unique_NAME_EDUCATION_TYPE_NAME_INCOME_TYPE', 'unique_NAME_EDUCATION_TYPE_x', 'unique_NAME_EDUCATION_TYPE_y', 'unique_NAME_INCOME_TYPE_x', 'unique_NAME_INCOME_TYPE_y']

In [ ]:
X_train = train[features]
X_test = test[features]

In [ ]:
X_train.shape, X_test.shape

In [ ]:
size = 3
window = 8

In [ ]:
x_w2v = copy.deepcopy(data[features])

In [ ]:
type(x_w2v)

In [ ]:
names = list(x_w2v.columns.values)
print(names)

In [ ]:
x_w2v.head(2)

In [ ]:
for i in names:
    x_w2v[i]=x_w2v[i].astype('category')
    x_w2v[i].cat.categories = ["Feature %s %s" % (i,g) for g in x_w2v[i].cat.categories]

In [ ]:
x_w2v.head(2)

In [ ]:
x_w2v = x_w2v.values.tolist()

In [ ]:
for i in x_w2v:
    shuffle(i)

In [ ]:
for col in features:
    print(data[col].nunique())

In [ ]:
w2v = Word2Vec(x_w2v,size=size,window=window)

In [ ]:
X_train_w2v = copy.copy(X_train)
X_test_w2v = copy.copy(X_test)

In [ ]:
X_train_w2v.head(2)

In [ ]:
for i in names:
    X_train_w2v[i]=X_train_w2v[i].astype('category')
    X_train_w2v[i].cat.categories = ["Feature %s %s" % (i,g) for g in X_train_w2v[i].cat.categories]

In [ ]:
X_train_w2v.head(2)

In [ ]:
for i in names:
    X_test_w2v[i]=X_test_w2v[i].astype('category')
    X_test_w2v[i].cat.categories = ["Feature %s %s" % (i,g) for g in X_test_w2v[i].cat.categories]
X_train_w2v = X_train_w2v.values
X_test_w2v = X_test_w2v.values

In [ ]:
x_w2v_train = np.random.random((len(X_train_w2v),size*X_train_w2v.shape[1]))
x_w2v_train.shape

In [ ]:
for j in range(X_train_w2v.shape[1]):
    for i in range(X_train_w2v.shape[0]):
        if X_train_w2v[i,j] in w2v.wv:
            x_w2v_train[i,j*size:(j+1)*size] = w2v.wv[X_train_w2v[i,j]]

In [ ]:
x_w2v_test = np.random.random((len(X_test_w2v),size*X_test_w2v.shape[1]))
print(x_w2v_test.shape)
for j in range(X_test_w2v.shape[1]):
    for i in range(X_test_w2v.shape[0]):
        if X_test_w2v[i,j] in w2v.wv:
            x_w2v_test[i,j*size:(j+1)*size] = w2v.wv[X_test_w2v[i,j]]

In [ ]:
test_cat_feats_embedded = pd.DataFrame(x_w2v_test)

In [ ]:
test_cat_feats_embedded.head(2)

In [ ]:
test_cat_feats_embedded.shape

In [ ]:
train_cat_feats_embedded = pd.DataFrame(x_w2v_train)

In [ ]:
train_cat_feats_embedded.shape

In [ ]:
print(train.shape)
train.drop(features, axis=1, inplace=True)
print(train.shape)

In [ ]:
train = pd.concat([train, train_cat_feats_embedded], axis=1)
train.shape

In [ ]:
print(test.shape)
test.drop(features, axis=1, inplace=True)
print(test.shape)

In [ ]:
test = pd.concat([test, test_cat_feats_embedded], axis=1)
test.shape

In [ ]:
train.to_pickle('data/train_bo2_w2v_newcat.pkl')

In [ ]:
test.to_pickle('data/test_bo2_w2v_newcat.pkl')